Where to find your access token :https://blogs.dropbox.com/developers/2014/05/generate-an-access-token-for-your-own-account/

In [1]:
import dropboxdrivefs as dbx
from fklab.io.cloud import dropboxAPI
import logging

In [14]:
db = dropboxAPI()
fs = dbx.DropboxDriveFileSystem(token=db.load_token())

## Create and remove a folder (files)

In [15]:
fs.mkdir("/test_dropbox")

INFO:Request to files/create_folder_v2
INFO:The directory/test_dropbox has  been created.


## Upload a file

In [16]:
fs.upload("test1.txt", "/test_dropbox/test1.txt")

INFO:Request to files/upload_session/start
INFO:Request to files/upload_session/finish


## Download a file

In [18]:
fs.download("/test_dropbox/test1.txt", "test1.txt")

INFO:Request to files/get_temporary_link
INFO:Request to files/get_metadata


## Write in a file

In [19]:
with fs.open("/test_dropbox/test.txt", mode='w') as f:
    f.write('hello')

INFO:Request to files/upload_session/start
INFO:Request to files/upload_session/finish


## Read a file 

In [20]:
file = fs.open("/test_dropbox/test.txt")

INFO:Request to files/get_temporary_link
INFO:Request to files/get_metadata


In [21]:
file.read()

b'hellohello'

## Read with intake

In [22]:
import intake

In [23]:
file_intake = intake.open_textfiles("dropbox:///test_dropbox/test.txt",storage_options={"token":db.load_token()})


In [24]:
file_intake.read()

INFO:Request to files/get_temporary_link
INFO:Request to files/get_metadata


['hellohello']

## Remove the folder

In [26]:
fs.rm("/test_dropbox")

INFO:Request to files/delete_v2
INFO:The directory/test_dropbox has been erased.
